In [14]:
import numpy as np
import pandas as pd

import scipy.sparse as sparce
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

import pickle
import nmslib

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [2]:
ratings = pd.read_csv("./data/ratings.csv")
books = pd.read_csv("./data/books.csv")
tags = pd.read_csv("./data/tags_cleaned.csv")
book_tags = pd.read_csv("./data/book_tags.csv")

In [3]:
books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [4]:
mapper = dict(zip(books.goodreads_book_id, books.book_id))


In [5]:
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: mapper[x])
book_tags.head()

,goodreads_book_id,tag_id,count,id
1,1,11305,37174,27
4,1,33114,12716,27
5,1,11743,9954,27
6,1,14017,7169,27
10,1,27199,3857,27


In [6]:
ratings_coo = sparce.coo_matrix((ratings.rating, (ratings.user_id, ratings.book_id)))


In [7]:
feature_ratings = sparce.coo_matrix(([1]*len(book_tags), (book_tags.id, book_tags.tag_id)))

In [8]:
NUM_THREADS = 3
NUM_COMPONENTS = 60
NUM_EPOCHS = 10

In [9]:
model = LightFM(learning_rate=0.05, loss='warp', no_components=NUM_COMPONENTS)
train, test = random_train_test_split(ratings_coo, test_percentage=0.2, random_state=42)
model = model.fit(train, epochs=NUM_EPOCHS, num_threads=NUM_THREADS, item_features=feature_ratings)

In [10]:
precision_score = precision_at_k(
    model,
    test,
    num_threads=NUM_THREADS,
    k=10,
    item_features=feature_ratings
).mean()
recall_score = recall_at_k(
    model,
    test,
    num_threads=NUM_THREADS,
    k=10,
    item_features=feature_ratings
).mean()

print(precision_score, recall_score)

0.08493319 0.03918657297570483


In [11]:
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
# Извлекаем эмбеддинги
item_biases, item_embeddings = model.get_item_representations(features=feature_ratings)

print(item_biases.shape, item_embeddings.shape)

(10001,) (10001, 60)


In [15]:
#Создаём граф для поиска
nms_ids = nmslib.init(method='hnsw', space='cosinesimil')

#Начинаем добавлять книги в граф
nms_ids.addDataPointBatch(item_embeddings)
nms_ids.createIndex(print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [16]:
def nearest_books_nms(book_id, index, n=10):
    nn = index.knnQuery(item_embeddings[book_id], k=n)
    
    return nn

In [18]:
books[books['title'].apply(lambda x: x.lower().find('1984')) >= 0]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
12,13,5470,5470,153313,995,451524934,9.780452e+12,"George Orwell, Erich Fromm, Celâl Üster",1949.0,Nineteen Eighty-Four,...,1956832,2053394,45518,41845,86425,324874,692021,908229,https://images.gr-assets.com/books/1348990566m...,https://images.gr-assets.com/books/1348990566s...
845,846,5472,5472,2966408,51,151010269,9.780151e+12,"George Orwell, Christopher Hitchens",1950.0,Animal Farm & 1984,...,116197,118761,1293,1212,3276,16511,40583,57179,https://images.gr-assets.com/books/1327959366m...,https://images.gr-assets.com/books/1327959366s...
9795,9796,201145,201145,2563528,25,64440508,9.780064e+12,"Else Holmelund Minarik, Maurice Sendak",1968.0,A Kiss for Little Bear,...,11063,11604,126,87,284,1898,3053,6282,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


In [19]:
nearest_books_nms(846, nms_ids)

(array([ 846,   14,   55,  809,   48,   13, 8140,  903,  271,  173],
       dtype=int32),
 array([1.1920929e-07, 3.4362018e-02, 3.9344609e-02, 5.6899190e-02,
        6.3437581e-02, 6.4570487e-02, 7.2078645e-02, 7.6902330e-02,
        9.0771079e-02, 9.1328979e-02], dtype=float32))

In [20]:
nmb = nearest_books_nms(846, nms_ids)[0]
nmb

array([ 846,   14,   55,  809,   48,   13, 8140,  903,  271,  173],
      dtype=int32)

In [22]:
books[books.book_id.isin(nmb)][['authors', 'title']]

,authors,title
12,"George Orwell, Erich Fromm, Celâl Üster",1984
13,George Orwell,Animal Farm
47,Ray Bradbury,Fahrenheit 451
54,Aldous Huxley,Brave New World
172,Anthony Burgess,A Clockwork Orange
270,Daniel Keyes,Flowers for Algernon
808,"Aldous Huxley, Christopher Hitchens",Brave New World / Brave New World Revisited
845,"George Orwell, Christopher Hitchens",Animal Farm / 1984
902,Ayn Rand,Anthem
8139,Aldous Huxley,Brave New World Revisited


In [ ]:
with open('item_embeddings.pkl', 'wb') as file:
    pickle.dump(item_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)